# ML based segmentations


## Functions 

In [6]:
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
import pandas as pd
from matplotlib import colors
import cv2
from skimage import measure

def make_bold(text):
    return f"\033[1m{text}\033[0m"
    
def normalize_image_to_uint8(image, lower_bound=-1000, upper_bound=100):
    clipped_img = np.clip(image, lower_bound, upper_bound)
    normalized_img = ((clipped_img - lower_bound) / (upper_bound - lower_bound)) * 255.0
    normalized_img = normalized_img.astype(np.uint8)
    return normalized_img



def find_nodule_lobe(cccwhd, lung_mask, class_map):
    """
    Determine the lung lobe where a nodule is located based on a 3D mask and bounding box.

    Parameters:
    cccwhd (list or tuple): Bounding box in the format [center_x, center_y, center_z, width, height, depth].
    lung_mask (numpy array): 3D array representing the lung mask with different lung regions.
    class_map (dict): Dictionary mapping lung region labels to their names.

    Returns:
    str: Name of the lung lobe where the nodule is located.
    """
    center_x, center_y, center_z, width, height, depth = cccwhd

    # Calculate the bounding box limits
    start_x = int(center_x - width // 2)
    end_x = int(center_x + width // 2)
    start_y = int(center_y - height // 2)
    end_y = int(center_y + height // 2)
    start_z = int(center_z - depth // 2)
    end_z = int(center_z + depth // 2)

    # Ensure the indices are within the mask dimensions
    start_x = max(0, start_x)
    end_x = min(lung_mask.shape[0], end_x)
    start_y = max(0, start_y)
    end_y = min(lung_mask.shape[1], end_y)
    start_z = max(0, start_z)
    end_z = min(lung_mask.shape[2], end_z)

    # Extract the region of interest (ROI) from the mask
    roi = lung_mask[start_x:end_x, start_y:end_y, start_z:end_z]

    # Count the occurrences of each lobe label within the ROI
    unique, counts = np.unique(roi, return_counts=True)
    label_counts = dict(zip(unique, counts))

    # Exclude the background (label 0)
    if 0 in label_counts:
        del label_counts[0]

    # Find the label with the maximum count
    if label_counts:
        nodule_lobe = max(label_counts, key=label_counts.get)
    else:
        nodule_lobe = None

    # Map the label to the corresponding lung lobe
    if nodule_lobe is not None:
        nodule_lobe_name = class_map["lungs"][nodule_lobe]
    else:
        nodule_lobe_name = "Undefined"

    return nodule_lobe_name



##------- Image Segmentation functions------##

import numpy as np
from sklearn.cluster import KMeans
import scipy.ndimage as ndimage
from sklearn.mixture import GaussianMixture
import skfuzzy as fuzz
from skimage.filters import threshold_otsu
from skimage.filters import threshold_otsu
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import morphology
from scipy.ndimage import distance_transform_edt, binary_erosion


def segment_nodule_kmeans(ct_image, bbox_center, bbox_whd, margin=5, n_clusters=2):
    """
    Segments a nodule in a 3D CT image using k-means clustering with a margin around the bounding box.

    Parameters:
    - ct_image: 3D NumPy array representing the CT image.
    - bbox_center: Tuple of (x, y, z) coordinates for the center of the bounding box.
    - bbox_whd: Tuple of (w, h, d) representing the width, height, and depth of the bounding box.
    - margin: Margin to add around the bounding box (default is 5).
    - n_clusters: Number of clusters to use in k-means (default is 2).

    Returns:
    - segmented_image: 3D NumPy array with the segmented nodule.
    """
    
    x_center, y_center, z_center = bbox_center
    w, h, d = bbox_whd

    # Calculate the bounding box with margin
    x_start, x_end = max(0, x_center - w//2 - margin), min(ct_image.shape[0], x_center + w//2 + margin)
    y_start, y_end = max(0, y_center - h//2 - margin), min(ct_image.shape[1], y_center + h//2 + margin)
    z_start, z_end = max(0, z_center - d//2 - margin), min(ct_image.shape[2], z_center + d//2 + margin)
    
    bbox_region = ct_image[x_start:x_end, y_start:y_end, z_start:z_end]

    # Reshape the region for k-means clustering
    flat_region = bbox_region.reshape(-1, 1)

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(flat_region)
    labels = kmeans.labels_

    # Reshape the labels back to the original bounding box shape
    clustered_region = labels.reshape(bbox_region.shape)

    # Assume the nodule is in the cluster with the highest mean intensity
    nodule_cluster = np.argmax(kmeans.cluster_centers_)

    # Create a binary mask for the nodule
    nodule_mask = (clustered_region == nodule_cluster)

    # Apply morphological operations to refine the segmentation
    nodule_mask = ndimage.binary_closing(nodule_mask, structure=np.ones((3, 3, 3)))
    nodule_mask = ndimage.binary_opening(nodule_mask, structure=np.ones((2, 2, 2)))

    # Initialize the segmented image
    segmented_image = np.zeros_like(ct_image, dtype=np.uint8)

    # Place the nodule mask in the correct position in the segmented image
    segmented_image[x_start:x_end, y_start:y_end, z_start:z_end] = nodule_mask

    return segmented_image




def segment_nodule_gmm(ct_image, bbox_center, bbox_whd, margin=5, n_components=2):
    """
    Segments a nodule in a 3D CT image using a Gaussian Mixture Model with a margin around the bounding box.

    Parameters:
    - ct_image: 3D NumPy array representing the CT image.
    - bbox_center: Tuple of (x, y, z) coordinates for the center of the bounding box.
    - bbox_whd: Tuple of (w, h, d) representing the width, height, and depth of the bounding box.
    - margin: Margin to add around the bounding box (default is 5).
    - n_components: Number of components to use in the Gaussian Mixture Model (default is 2).

    Returns:
    - segmented_image: 3D NumPy array with the segmented nodule.
    """
    
    x_center, y_center, z_center = bbox_center
    w, h, d = bbox_whd

    # Calculate the bounding box with margin
    x_start, x_end = max(0, x_center - w//2 - margin), min(ct_image.shape[0], x_center + w//2 + margin)
    y_start, y_end = max(0, y_center - h//2 - margin), min(ct_image.shape[1], y_center + h//2 + margin)
    z_start, z_end = max(0, z_center - d//2 - margin), min(ct_image.shape[2], z_center + d//2 + margin)
    
    bbox_region = ct_image[x_start:x_end, y_start:y_end, z_start:z_end]

    # Reshape the region for GMM
    flat_region = bbox_region.reshape(-1, 1)

    # Perform GMM
    gmm = GaussianMixture(n_components=n_components, random_state=0).fit(flat_region)
    labels = gmm.predict(flat_region)

    # Reshape the labels back to the original bounding box shape
    clustered_region = labels.reshape(bbox_region.shape)

    # Assume the nodule is in the component with the highest mean intensity
    nodule_component = np.argmax(gmm.means_)

    # Create a binary mask for the nodule
    nodule_mask = (clustered_region == nodule_component)

    # Apply morphological operations to refine the segmentation
    nodule_mask = ndimage.binary_closing(nodule_mask, structure=np.ones((3, 3, 3)))
    nodule_mask = ndimage.binary_opening(nodule_mask, structure=np.ones((3, 3, 3)))

    # Initialize the segmented image
    segmented_image = np.zeros_like(ct_image, dtype=np.uint8)

    # Place the nodule mask in the correct position in the segmented image
    segmented_image[x_start:x_end, y_start:y_end, z_start:z_end] = nodule_mask

    return segmented_image





def segment_nodule_fcm(ct_image, bbox_center, bbox_whd, margin=5, n_clusters=2):
    """
    Segments a nodule in a 3D CT image using Fuzzy C-means clustering with a margin around the bounding box.

    Parameters:
    - ct_image: 3D NumPy array representing the CT image.
    - bbox_center: Tuple of (x, y, z) coordinates for the center of the bounding box.
    - bbox_whd: Tuple of (w, h, d) representing the width, height, and depth of the bounding box.
    - margin: Margin to add around the bounding box (default is 5).
    - n_clusters: Number of clusters to use in Fuzzy C-means (default is 2).

    Returns:
    - segmented_image: 3D NumPy array with the segmented nodule.
    """
    
    x_center, y_center, z_center = bbox_center
    w, h, d = bbox_whd

    # Calculate the bounding box with margin
    x_start, x_end = max(0, x_center - w//2 - margin), min(ct_image.shape[0], x_center + w//2 + margin)
    y_start, y_end = max(0, y_center - h//2 - margin), min(ct_image.shape[1], y_center + h//2 + margin)
    z_start, z_end = max(0, z_center - d//2 - margin), min(ct_image.shape[2], z_center + d//2 + margin)
    
    bbox_region = ct_image[x_start:x_end, y_start:y_end, z_start:z_end]

    # Reshape the region for FCM
    flat_region = bbox_region.reshape(-1, 1)

    # Perform FCM clustering
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(flat_region.T, n_clusters, 2, error=0.005, maxiter=1000, init=None)
    
    # Assign each voxel to the cluster with the highest membership
    labels = np.argmax(u, axis=0)

    # Reshape the labels back to the original bounding box shape
    clustered_region = labels.reshape(bbox_region.shape)

    # Assume the nodule is in the cluster with the highest mean intensity
    nodule_cluster = np.argmax(cntr)

    # Create a binary mask for the nodule
    nodule_mask = (clustered_region == nodule_cluster)

    # Apply morphological operations to refine the segmentation
    nodule_mask = ndimage.binary_closing(nodule_mask, structure=np.ones((3, 3, 3)))
    nodule_mask = ndimage.binary_opening(nodule_mask, structure=np.ones((3, 3, 3)))

    # Initialize the segmented image
    segmented_image = np.zeros_like(ct_image, dtype=np.uint8)

    # Place the nodule mask in the correct position in the segmented image
    segmented_image[x_start:x_end, y_start:y_end, z_start:z_end] = nodule_mask

    return segmented_image



def segment_nodule_otsu(ct_image, bbox_center, bbox_whd, margin=5):
    """
    Segments a nodule in a 3D CT image using Otsu's thresholding with a margin around the bounding box.

    Parameters:
    - ct_image: 3D NumPy array representing the CT image.
    - bbox_center: Tuple of (x, y, z) coordinates for the center of the bounding box.
    - bbox_whd: Tuple of (w, h, d) representing the width, height, and depth of the bounding box.
    - margin: Margin to add around the bounding box (default is 5).

    Returns:
    - segmented_image: 3D NumPy array with the segmented nodule.
    """
    
    x_center, y_center, z_center = bbox_center
    w, h, d = bbox_whd

    # Calculate the bounding box with margin
    x_start, x_end = max(0, x_center - w//2 - margin), min(ct_image.shape[0], x_center + w//2 + margin)
    y_start, y_end = max(0, y_center - h//2 - margin), min(ct_image.shape[1], y_center + h//2 + margin)
    z_start, z_end = max(0, z_center - d//2 - margin), min(ct_image.shape[2], z_center + d//2 + margin)
    
    bbox_region = ct_image[x_start:x_end, y_start:y_end, z_start:z_end]

    # Flatten the region for thresholding
    flat_region = bbox_region.flatten()

    # Calculate the Otsu threshold
    otsu_threshold = threshold_otsu(flat_region)

    # Apply the threshold to create a binary mask
    nodule_mask = bbox_region >= otsu_threshold

    # Apply morphological operations to refine the segmentation
    nodule_mask = ndimage.binary_closing(nodule_mask, structure=np.ones((3, 3, 3)))
    nodule_mask = ndimage.binary_opening(nodule_mask, structure=np.ones((3, 3, 3)))

    # Initialize the segmented image
    segmented_image = np.zeros_like(ct_image, dtype=np.uint8)

    # Place the nodule mask in the correct position in the segmented image
    segmented_image[x_start:x_end, y_start:y_end, z_start:z_end] = nodule_mask

    return segmented_image







def segment_nodule_watershed(ct_image, bbox_center, bbox_whd, margin=5):
    """
    Segments a nodule in a 3D CT image using the watershed algorithm with Otsu's thresholding.

    Parameters:
    - ct_image: 3D NumPy array representing the CT image.
    - bbox_center: Tuple of (x, y, z) coordinates for the center of the bounding box.
    - bbox_whd: Tuple of (w, h, d) representing the width, height, and depth of the bounding box.
    - margin: Margin to add around the bounding box (default is 5).

    Returns:
    - segmented_image: 3D NumPy array with the segmented nodule.
    """
    
    x_center, y_center, z_center = bbox_center
    w, h, d = bbox_whd

    # Calculate the bounding box with margin
    x_start, x_end = max(0, x_center - w//2 - margin), min(ct_image.shape[0], x_center + w//2 + margin)
    y_start, y_end = max(0, y_center - h//2 - margin), min(ct_image.shape[1], y_center + h//2 + margin)
    z_start, z_end = max(0, z_center - d//2 - margin), min(ct_image.shape[2], z_center + d//2 + margin)
    
    bbox_region = ct_image[x_start:x_end, y_start:y_end, z_start:z_end]

    # Flatten the region for thresholding
    flat_region = bbox_region.flatten()

    # Calculate the Otsu threshold
    otsu_threshold = threshold_otsu(flat_region)

    # Apply the threshold to create a binary mask
    binary_mask = bbox_region >= otsu_threshold

    # Compute the distance transform
    distance = ndimage.distance_transform_edt(binary_mask)

    # Find local maxima
    local_maxi = peak_local_max(distance, footprint=np.ones((3, 3, 3)), labels=binary_mask)

    # Markers for the watershed
    markers = ndimage.label(local_maxi)[0]

    # Apply watershed
    labels = watershed(-distance, markers, mask=binary_mask)

    # Create a binary mask for the nodule
    nodule_mask = labels > 0

    # Apply morphological operations to refine the segmentation
    nodule_mask = ndimage.binary_closing(nodule_mask, structure=np.ones((3, 3, 3)))
    nodule_mask = ndimage.binary_opening(nodule_mask, structure=np.ones((3, 3, 3)))

    # Initialize the segmented image
    segmented_image = np.zeros_like(ct_image, dtype=np.uint8)

    # Place the nodule mask in the correct position in the segmented image
    segmented_image[x_start:x_end, y_start:y_end, z_start:z_end] = nodule_mask

    return segmented_image


import numpy as np
from scipy.ndimage import distance_transform_edt

def find_nodule_lobe_and_distance(cccwhd, lung_mask, class_map,spacing):
    """
    Determine the lung lobe where a nodule is located and measure its distance from the lung wall.

    Parameters:
    cccwhd (list or tuple): Bounding box in the format [center_x, center_y, center_z, width, height, depth].
    lung_mask (numpy array): 3D array representing the lung mask with different lung regions.
    class_map (dict): Dictionary mapping lung region labels to their names.

    Returns:
    tuple: (Name of the lung lobe, Distance from the lung wall)
    """
    center_x, center_y, center_z, width, height, depth = cccwhd

    # Calculate the bounding box limits
    start_x = int(center_x - width // 2)
    end_x = int(center_x + width // 2)
    start_y = int(center_y - height // 2)
    end_y = int(center_y + height // 2)
    start_z = int(center_z - depth // 2)
    end_z = int(center_z + depth // 2)

    # Ensure the indices are within the mask dimensions
    start_x = max(0, start_x)
    end_x = min(lung_mask.shape[0], end_x)
    start_y = max(0, start_y)
    end_y = min(lung_mask.shape[1], end_y)
    start_z = max(0, start_z)
    end_z = min(lung_mask.shape[2], end_z)

    # Extract the region of interest (ROI) from the mask
    roi = lung_mask[start_x:end_x, start_y:end_y, start_z:end_z]

    # Count the occurrences of each lobe label within the ROI
    unique, counts = np.unique(roi, return_counts=True)
    label_counts = dict(zip(unique, counts))

    # Exclude the background (label 0)
    if 0 in label_counts:
        del label_counts[0]

    # Find the label with the maximum count
    if label_counts:
        nodule_lobe = max(label_counts, key=label_counts.get)
    else:
        nodule_lobe = None

    # Map the label to the corresponding lung lobe
    if nodule_lobe is not None:
        nodule_lobe_name = class_map["lungs"][nodule_lobe]
    else:
        nodule_lobe_name = "Undefined"

    # Calculate the distance from the nodule centroid to the nearest lung wall
    nodule_centroid = np.array([center_x, center_y, center_z])
    
    # Create a binary lung mask where lung region is 1 and outside lung is 0
    lung_binary_mask = lung_mask > 0

    # Create the lung wall mask by finding the outer boundary
    # Use binary erosion to shrink the lung mask, then subtract it from the original mask to get the boundary
    lung_eroded    = binary_erosion(lung_binary_mask)
    lung_wall_mask = lung_binary_mask & ~lung_eroded  # Lung wall mask is the outermost boundary (contour)

    # Compute the distance transform from the lung wall
    distance_transform = distance_transform_edt(~lung_wall_mask)  # Compute distance to nearest lung boundary

    
    
    # Get the distance from the nodule centroid to the nearest lung wall in voxel units
    voxel_distance_to_lung_wall = distance_transform[center_x, center_y, center_z]

    # Convert voxel distance to real-world distance in mm
    physical_distance_to_lung_wall = voxel_distance_to_lung_wall * np.sqrt(
        spacing[0]**2 + spacing[1]**2 + spacing[2]**2
    )



    return nodule_lobe_name, voxel_distance_to_lung_wall,physical_distance_to_lung_wall


def expand_mask_by_distance(segmented_nodule_gmm, spacing, expansion_mm):
    """
    Expands the segmentation mask by a given distance in mm in all directions by directly updating pixel values.

    Parameters:
    segmented_nodule_gmm (numpy array): 3D binary mask of the nodule (1 for nodule, 0 for background).
    spacing (tuple): Spacing of the image in mm for each voxel, given as (spacing_x, spacing_y, spacing_z).
    expansion_mm (float): Distance to expand the mask in millimeters.

    Returns:
    numpy array: Expanded segmentation mask.
    """
    # Reorder spacing to match the numpy array's (z, y, x) format
    spacing_reordered = (spacing[2], spacing[1], spacing[0])  # (spacing_z, spacing_y, spacing_x)

    # Calculate the number of pixels to expand in each dimension
    expand_pixels = np.array([int(np.round(expansion_mm / s)) for s in spacing_reordered])

    # Create a new expanded mask with the same shape
    expanded_mask = np.zeros_like(segmented_nodule_gmm)

    # Get the coordinates of all white pixels in the original mask
    white_pixel_coords = np.argwhere(segmented_nodule_gmm == 1)

    # Expand each white pixel by adding the specified number of pixels in each direction
    for coord in white_pixel_coords:
        z, y, x = coord  # Extract the z, y, x coordinates of each white pixel

        # Define the range to expand for each coordinate
        z_range = range(max(0, z - expand_pixels[0]), min(segmented_nodule_gmm.shape[0], z + expand_pixels[0] + 1))
        y_range = range(max(0, y - expand_pixels[1]), min(segmented_nodule_gmm.shape[1], y + expand_pixels[1] + 1))
        x_range = range(max(0, x - expand_pixels[2]), min(segmented_nodule_gmm.shape[2], x + expand_pixels[2] + 1))

        # Update the new mask by setting all pixels in this range to 1
        for z_new in z_range:
            for y_new in y_range:
                for x_new in x_range:
                    expanded_mask[z_new, y_new, x_new] = 1

    return expanded_mask




# Function to plot the contours of a mask
def plot_contours(ax, mask, color, linewidth=1.5):
    contours = measure.find_contours(mask, level=0.5)  # Find contours at a constant level
    for contour in contours:
        ax.plot(contour[:, 1], contour[:, 0], color=color, linewidth=linewidth)

# Just getting lob-loc( the location)

In [1]:
import sys
sys.path.append('/data/usr/ft42/CVIT_XAI/LungRADS_Modeling/Nodule_Segmentation/')  # Replace with the actual path
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.cluster")

from segmentation_eval_utils import*
import pandas as pd
import SimpleITK as sitk
from sklearn.metrics import jaccard_score, precision_score, recall_score
import math

###############################################

def nodule_diameter(H, W, D):
    """
    Calculate the effective diameter of a 3D nodule given height, width, and depth.

    Parameters:
    H (float): Height of the nodule
    W (float): Width of the nodule
    D (float): Depth of the nodule

    Returns:
    float: Effective diameter
    """
    return (H * W * D) ** (1/3)


def nodule_effective_diameter(H, W, D):
    volume = (4/3) * math.pi * (H/2) * (W/2) * (D/2)
    return 2 * ((3 / (4 * math.pi)) * volume) ** (1/3)

def compute_iou(gt_mask, pred_mask):
    """Calculate Intersection over Union (IoU)."""
    intersection = np.logical_and(gt_mask, pred_mask).sum()
    union = np.logical_or(gt_mask, pred_mask).sum()
    return intersection / union if union != 0 else 0.0

def compute_dice(gt_mask, pred_mask):
    """Calculate Dice Similarity Coefficient."""
    intersection = np.logical_and(gt_mask, pred_mask).sum()
    return (2.0 * intersection) / (gt_mask.sum() + pred_mask.sum()) if (gt_mask.sum() + pred_mask.sum()) != 0 else 0.0

def compute_performances(gt, pred):
    iou       = compute_iou(gt, pred)
    dice      = compute_dice(gt, pred)
    return iou, dice

##############################################
raw_data_path                   = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/luna25_images/'
dataset_csv                     = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/LUNA25_Public_Training_Development_Data_AnnotationID_nifti.csv'
which_seg_alg                   = 'knn'
vista3D_alg                     = True
knn_alg_3view_plot              = False
vista3D_alg_3view_plot          = False
SegModels_comparison_plot       = False
comparison_plot                 = True
view_3D_Nodule                  = True
view_margin                     = 24
compute_overall_dice            = True
#---- Mask_paths---
DukeSeg_raw_lungSeg_dir         = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/Vista3D_auto_eval_LUNA25tr_allseg/'
vista3D_OrangeSeg_dir           = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/Vista3D_auto_eval_LUNA25tr_allseg/'
vista3D_NoduleSeg_dir           = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/Vista3D_auto_eval_LUNA25tr_noduleseg/'
vista3D_NoduleSeg_t604_dir      = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/vista3d_t604_LUNA25tr_Test_noduleseg/'
vista3D_NoduleSeg_t600_dir      = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/vista3d_t501_LUNA25tr_Test_noduleseg/'
nnUNet_NodulennUNET604_dir      = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/nnUNet_Dataset604_NoduleSeg_LUNA25tr/'
nnUNet_NodulennUNET600_dir      = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/nnUNet_Dataset501_ULNoduleSeg_LUNA25tr/'
nnUNet_NodulennUNET602_dir      = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/nnUNet_Dataset602_NoduleSeg_LUNA25tr/'
vista3D_NoduleSeg_t602_dir      = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/Segmentation_results/vista3d_t602_LUNA25tr_Test_noduleseg/'

class_map = {"lungs": {
        1: "lung_upper_lobe_left",
        2: "lung_lower_lobe_left",
        3: "lung_upper_lobe_right",
        4: "lung_middle_lobe_right",
        5: "lung_lower_lobe_right"}}
class_map_Vista3d = {"lungs": {
        28: "lung_upper_lobe_left",
        29: "lung_lower_lobe_left",
        30: "lung_upper_lobe_right",
        31: "lung_middle_lobe_right",
        32: "lung_lower_lobe_right"}}


colors_list       = ['black', 'blue', 'yellow', 'purple', 'purple','purple']
cmap              = colors.ListedColormap(colors_list)
bounds            = [0, 1, 2, 3, 4, 5, 6]
norm              = colors.BoundaryNorm(bounds, cmap.N)


gt_box_clr                    = 'yellow'
Vista3DnoduleSeg_Mask_clr     = 'cyan'
Vista3DNoduleSegt600_Mask_clr = '#8BCB8F'
Vista3DNoduleSegt604_Mask_clr = '#D96F6F'
Vista3DNodulennUNET600_Mask_clr = '#708238'
Vista3DNodulennUNET604_Mask_clr = 'pink'
Vista3DNodulennUNET602_Mask_clr = 'purple'
Vista3DNoduleSegt602_Mask_clr = 'orange'















model_colors = {
    'Vista3D auto': 'cyan',
    'Vista3D auto: Finetuned Clinical': '#8BCB8F',
    'Vista3D auto: Finetuned Clinical + Simulated': '#D96F6F',
    'nnUNet Clinical': '#708238',
    'nnUNet Clinical + Simulated': 'pink'
}


##############################################



df         = pd.read_csv(dataset_csv)
final_dict = df['SeriesInstanceUID'].unique()
#final_dict = ['CT_CTGTFZ_TMP407292_302.nii.gz']

nodule_location    = []
nodule_distance_from_the_wall    = []
nodule_location_id = []
results_list = []
Overall_dice_result = []
problamatic_List =[]
# Loop to process each unique CT file
for idx in range(0,len(final_dict)):
    try:
        # Processing each image
        ct_path             = raw_data_path + final_dict[idx]+'.mha'
        ct_image            = sitk.ReadImage(ct_path)
        ct_array            = sitk.GetArrayFromImage(ct_image)
        filtered_df= df[df['SeriesInstanceUID'] == final_dict[idx]].reset_index()


        lung_mask_path        = vista3D_OrangeSeg_dir+'luna25_'+filtered_df['SeriesInstanceUID'][0]+'_0000'+'/'+'luna25_'+filtered_df['SeriesInstanceUID'][0]+'_0000_trans.nii.gz'
        ctlungmask_image      = sitk.ReadImage(lung_mask_path)
        ctlungmask_array      = sitk.GetArrayFromImage(ctlungmask_image)

        creating_lung_mask = np.zeros_like(ctlungmask_array)
        creating_lung_mask[ctlungmask_array==28] = 1
        creating_lung_mask[ctlungmask_array==29] = 2
        creating_lung_mask[ctlungmask_array==30] = 3
        creating_lung_mask[ctlungmask_array==31] = 4
        creating_lung_mask[ctlungmask_array==32] = 5


        #print('|------------------------ No={} ---------------------|'.format(dictonary_list_i))
        print(make_bold('|' + '-'*30 + ' No={} '.format(idx+1) + '-'*30 + '|'))
        print('\n')
        #print('Loading the Image:{}'.format(filtered_df['SeriesInstanceUID'][0]))
        #print('Loading the lung Mask:{}'.format(lung_mask_path))
        #print('Number of Annotation:{}'.format(len(filtered_df)))

        over_all_gt = []
        for j, row in filtered_df.iterrows():
            
            print('-----------------------------------------------------------------------------------------------')
            print('Annotation-ID ={}'.format(row["AnnotationID"]))
            world_coords = np.asarray([float(row["CoordX"]), float(row["CoordY"]), float(row["CoordZ"])])
            voxel_coords = ct_image.TransformPhysicalPointToIndex(world_coords)
            #print('WorlCoord  CCC (x,y,z) ={}'.format(world_coords))
            #print('VoxelCoord CCC (x,y,z) ={}'.format(voxel_coords))


            assuming_diameter = 8
            # Calculate bounding box dimensions
            whd_worldCoord  = np.asarray([float(assuming_diameter),float(assuming_diameter),float(assuming_diameter)])
            spacing         = ct_image.GetSpacing()
            w = int(float(assuming_diameter) / spacing[0])
            h = int(float(assuming_diameter) / spacing[1])
            d = int(float(assuming_diameter) / spacing[2])
            whd_voxelCoord  = [w,h,d]

            print('WorldCoord (w,h,d) ={}'.format(whd_worldCoord))
            print('VoxelCoord (w,h,d) ={}'.format(whd_voxelCoord))
            
            Calculated_Diameter=nodule_effective_diameter(H=whd_worldCoord[1], W=whd_worldCoord[0], D=whd_worldCoord[2])
            print('Diameter mm ={}'.format(Calculated_Diameter))


            nodule_lobe_name = find_nodule_lobe(cccwhd=[voxel_coords[2], voxel_coords[1],   voxel_coords[0],
                                                           whd_voxelCoord[2],whd_voxelCoord[1],whd_voxelCoord[0]], 
                                                           lung_mask=creating_lung_mask, class_map=class_map)
            print(f'The nodule belongs to: {nodule_lobe_name}')            
            nodule_location.append(nodule_lobe_name)
            nodule_location_id.append(row["AnnotationID"])



    except:
        print('problem-in this case')
        problamatic_List.append(row["AnnotationID"])
        pass

        

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


|------------------------------ No=1 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =100570_1_19990102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[12, 12, 4]
Diameter mm =7.999999999999999
The nodule belongs to: Undefined
|------------------------------ No=2 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =106364_1_19990102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[15, 15, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_upper_lobe_left
|------------------------------ No=3 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =127302_1_20000102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[11, 11, 3]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lower_lobe_ri

|------------------------------ No=22 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =117502_1_20010102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[12, 12, 4]
Diameter mm =7.999999999999999
The nodule belongs to: Undefined
|------------------------------ No=23 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =117612_1_19990102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[12, 12, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_upper_lobe_right
|------------------------------ No=24 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =117612_1_20000102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[12, 12, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_upper_lob

|------------------------------ No=45 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =212068_1_20000102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[10, 10, 9]
Diameter mm =7.999999999999999
The nodule belongs to: lung_upper_lobe_left
|------------------------------ No=46 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =212068_1_20010102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[12, 12, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_upper_lobe_left
|------------------------------ No=47 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =107561_1_19990102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[10, 10, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung

|------------------------------ No=69 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =212587_1_20010102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[12, 12, 6]
Diameter mm =7.999999999999999
The nodule belongs to: lung_upper_lobe_left
|------------------------------ No=70 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =119212_1_20010102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[10, 10, 4]
Diameter mm =7.999999999999999
The nodule belongs to: Undefined
|------------------------------ No=71 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =205089_1_19990102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[10, 10, 9]
Diameter mm =7.999999999999999
The nodule belongs to: lung_upper_lobe

...

|------------------------------ No=4000 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =213240_2_20010102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[13, 13, 6]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lower_lobe_right
-----------------------------------------------------------------------------------------------
Annotation-ID =213240_1_20010102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[13, 13, 6]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lower_lobe_left
|------------------------------ No=4001 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =118272_1_19990102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[12, 12, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_upper_lobe_right
|------------------------------ No=4002 ---------

|------------------------------ No=4019 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =129479_4_20010102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[11, 11, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_middle_lobe_right
-----------------------------------------------------------------------------------------------
Annotation-ID =129479_6_20010102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[11, 11, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lower_lobe_right
-----------------------------------------------------------------------------------------------
Annotation-ID =129479_3_20010102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[11, 11, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lower_lobe_left
-----------------------------------------------------------------------------------------------
Annotation-ID =129479_1_20

|------------------------------ No=4036 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =200942_1_19990102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[12, 12, 8]
Diameter mm =7.999999999999999
The nodule belongs to: Undefined
|------------------------------ No=4037 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =114689_1_19990102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[14, 14, 3]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lower_lobe_left
|------------------------------ No=4038 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =114689_3_20000102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[14, 14, 3]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lowe

|------------------------------ No=4052 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =111668_1_19990102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[13, 13, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lower_lobe_right
|------------------------------ No=4053 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =111668_4_20000102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[14, 14, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_upper_lobe_right
-----------------------------------------------------------------------------------------------
Annotation-ID =111668_1_20000102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[14, 14, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lower_lobe_right
------------------------------------------------

|------------------------------ No=4069 ------------------------------|


-----------------------------------------------------------------------------------------------
Annotation-ID =204005_1_20000102
WorldCoord (w,h,d) =[8. 8. 8.]
VoxelCoord (w,h,d) =[12, 12, 4]
Diameter mm =7.999999999999999
The nodule belongs to: lung_lower_lobe_left


In [2]:
problamatic_List

[]

In [5]:
luna25_lobe_loc_df = pd.DataFrame(list(zip(nodule_location_id,nodule_location)),columns=['AnnotationID','lob_loc'])
luna25_lobe_loc_df.to_csv('/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/LUNA25_Public_Training_Development_Data_AnnotationID_lobloc.csv',index=False,encoding='utf-8')


# Getting Slice Number 

In [9]:
import sys
sys.path.append('/data/usr/ft42/CVIT_XAI/LungRADS_Modeling/Nodule_Segmentation/')  # Replace with the actual path
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.cluster")

from segmentation_eval_utils import*
import pandas as pd
import SimpleITK as sitk
from sklearn.metrics import jaccard_score, precision_score, recall_score
import math

###############################################
##############################################
raw_data_path                   = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/luna25_images/'
dataset_csv                     = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/LUNA25_Public_Training_Development_Data_AnnotationID_nifti.csv'






##############################################



df         = pd.read_csv(dataset_csv)
final_dict = df['SeriesInstanceUID'].unique()
#final_dict = ['CT_CTGTFZ_TMP407292_302.nii.gz']

Nodule_Annotationid =[]
Nodule_Slice_Number =[]
Nodule_Slice_imgCoordX =[]
Nodule_Slice_imgCoordY =[]
# Loop to process each unique CT file
for idx in range(0,len(final_dict)):
    try:
        # Processing each image
        ct_path             = raw_data_path + final_dict[idx]+'.mha'
        ct_image            = sitk.ReadImage(ct_path)
        ct_array            = sitk.GetArrayFromImage(ct_image)
        filtered_df= df[df['SeriesInstanceUID'] == final_dict[idx]].reset_index()

        #print('|------------------------ No={} ---------------------|'.format(dictonary_list_i))
        print(make_bold('|' + '-'*30 + ' No={} '.format(idx+1) + '-'*30 + '|'))


        over_all_gt = []
        for j, row in filtered_df.iterrows():
            
            print('-----------------------------------------------------------------------------------------------')
            print('Annotation-ID ={}'.format(row["AnnotationID"]))
            world_coords = np.asarray([float(row["CoordX"]), float(row["CoordY"]), float(row["CoordZ"])])
            voxel_coords = ct_image.TransformPhysicalPointToIndex(world_coords)
            Nodule_Annotationid.append(row["AnnotationID"])
            Nodule_Slice_Number.append(voxel_coords[2])
            Nodule_Slice_imgCoordX.append(voxel_coords[0])
            Nodule_Slice_imgCoordY.append(voxel_coords[1])

    except:
        print('problem-in this case')
        pass



luna25_imgCoord_df = pd.DataFrame(list(zip(Nodule_Annotationid,Nodule_Slice_imgCoordX,Nodule_Slice_imgCoordY,Nodule_Slice_Number)),columns=['AnnotationID','imgCoordX','imgCoordZ','imgCoordZ'])
luna25_imgCoord_df.to_csv('/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/LUNA25_Public_Training_Development_Data_AnnotationID_ImgCoordD_SliceN.csv',index=False,encoding='utf-8')




|------------------------------ No=1 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =100570_1_19990102
|------------------------------ No=2 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =106364_1_19990102
|------------------------------ No=3 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =127302_1_20000102
|------------------------------ No=4 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =128899_1_20010102
|------------------------------ No=5 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =215816_1_19990102
----------

|------------------------------ No=37 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =218499_1_19990102
|------------------------------ No=38 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =218499_1_20000102
|------------------------------ No=39 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =218499_1_20010102
|------------------------------ No=40 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =117627_1_19990102
|------------------------------ No=41 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =119496_1_19990102
|----

|------------------------------ No=77 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =101287_1_19990102
|------------------------------ No=78 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =132227_1_19990102
|------------------------------ No=79 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =202715_1_19990102
|------------------------------ No=80 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =202715_1_20000102
|------------------------------ No=81 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =104363_1_19990102
|----

|------------------------------ No=115 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =118456_1_19990102
|------------------------------ No=116 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =118456_1_20000102
|------------------------------ No=117 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =118456_1_20010102
|------------------------------ No=118 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =127121_1_20010102
|------------------------------ No=119 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =115979_3_19990102


|------------------------------ No=146 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =215245_2_20010102
|------------------------------ No=147 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =103786_1_19990102
-----------------------------------------------------------------------------------------------
Annotation-ID =103786_2_19990102
|------------------------------ No=148 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =103786_5_20000102
-----------------------------------------------------------------------------------------------
Annotation-ID =103786_4_20000102
-----------------------------------------------------------------------------------------------
Annotation-ID =103786_1_20000102
|------------

...

|------------------------------ No=3929 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =101143_1_20010102
|------------------------------ No=3930 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =131520_1_19990102
-----------------------------------------------------------------------------------------------
Annotation-ID =131520_6_19990102
-----------------------------------------------------------------------------------------------
Annotation-ID =131520_2_19990102
-----------------------------------------------------------------------------------------------
Annotation-ID =131520_3_19990102
|------------------------------ No=3931 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =131520_1_20000102
----------

|------------------------------ No=3960 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =120447_1_19990102
|------------------------------ No=3961 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =120447_1_20000102
|------------------------------ No=3962 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =120447_1_20010102
|------------------------------ No=3963 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =206874_1_19990102
|------------------------------ No=3964 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =108310_1_1999

|------------------------------ No=3990 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =214058_2_19990102
-----------------------------------------------------------------------------------------------
Annotation-ID =214058_1_19990102
|------------------------------ No=3991 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =214058_2_20000102
-----------------------------------------------------------------------------------------------
Annotation-ID =214058_3_20000102
-----------------------------------------------------------------------------------------------
Annotation-ID =214058_1_20000102
|------------------------------ No=3992 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =214058_1_20010102
|---------

|------------------------------ No=4019 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =129479_4_20010102
-----------------------------------------------------------------------------------------------
Annotation-ID =129479_6_20010102
-----------------------------------------------------------------------------------------------
Annotation-ID =129479_3_20010102
-----------------------------------------------------------------------------------------------
Annotation-ID =129479_1_20010102
-----------------------------------------------------------------------------------------------
Annotation-ID =129479_7_20010102
|------------------------------ No=4020 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =104864_2_19990102
----------------------------------------------------------------------------------

|------------------------------ No=4046 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =123400_1_20000102
-----------------------------------------------------------------------------------------------
Annotation-ID =123400_2_20000102
|------------------------------ No=4047 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =123400_1_20010102
-----------------------------------------------------------------------------------------------
Annotation-ID =123400_2_20010102
|------------------------------ No=4048 ------------------------------|
-----------------------------------------------------------------------------------------------
Annotation-ID =115985_1_19990102
|------------------------------ No=4049 ------------------------------|
-------------------------------------------------------------------

In [11]:
luna25_imgCoord_df = pd.DataFrame(list(zip(Nodule_Annotationid,Nodule_Slice_imgCoordX,Nodule_Slice_imgCoordY,Nodule_Slice_Number)),columns=['AnnotationID','imgCoordX','imgCoordY','imgCoordZ'])
luna25_imgCoord_df.to_csv('/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/LUNA25_Public_Training_Development_Data_AnnotationID_ImgCoordD_SliceN.csv',index=False,encoding='utf-8')
luna25_imgCoord_df


,AnnotationID,imgCoordX,imgCoordY,imgCoordZ
0,100570_1_19990102,352,276,88
1,106364_1_19990102,328,336,110
2,127302_1_20000102,141,243,49
3,128899_1_20010102,214,261,90
4,215816_1_19990102,321,222,93
...,...,...,...,...
6158,123244_1_20010102,307,235,117
6159,104196_1_20000102,390,356,137
6160,104196_1_20010102,372,345,135
6161,204005_1_19990102,445,274,111


In [1]:
import pandas as pd
raw_data_path                   = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/luna25_images/'
dataset_csv                     = '/NAS/shared_data/for_VNLST/ft42/ct_public/LUNA25/LUNA25_Public_Training_Development_Data_AnnotationID_nifti.csv'


df               = pd.read_csv(dataset_csv)
unique_patientid = df['PatientID'].unique()
unique_patientid_df = df.drop_duplicates(subset='PatientID', keep='first')

print('##--| Counts')
print('Unique Patient Counts:{}| Nodule Annoation:{}'.format(len(unique_patientid),len(df)))

print('##--| Gender')
print(unique_patientid_df['Gender'].value_counts())
print(unique_patientid_df['Gender'].value_counts(True))

print(unique_patientid_df['Age_at_StudyDate'].describe())

print(unique_patientid_df['label'].value_counts())
print(unique_patientid_df['label'].value_counts(True))

##--| Counts
Unique Patient Counts:2120| Nodule Annoation:6163
##--| Gender
Gender
Male      1211
Female     909
Name: count, dtype: int64
Gender
Male      0.571226
Female    0.428774
Name: proportion, dtype: float64
count    2120.000000
mean       62.745283
std         5.147125
min        55.000000
25%        58.000000
50%        62.000000
75%        67.000000
max        76.000000
Name: Age_at_StudyDate, dtype: float64
label
0    1787
1     333
Name: count, dtype: int64
label
0    0.842925
1    0.157075
Name: proportion, dtype: float64


In [11]:
print(df['label'].value_counts())
print(df['label'].value_counts(True))

0    5608
1     555
Name: label, dtype: int64
0    0.909946
1    0.090054
Name: label, dtype: float64


In [13]:
unique_ct = df.drop_duplicates(subset='SeriesInstanceUID', keep='first')

In [14]:
len(unique_ct)

4069

In [15]:
len(df)

6163

In [2]:
df

,PatientID,SeriesInstanceUID,StudyDate,CoordX,CoordY,CoordZ,LesionID,AnnotationID,NoduleID,label,Age_at_StudyDate,Gender,AnnotationID_nifti
0,100570,1.2.840.113654.2.55.32380467633296345717423514...,19990102,61.14,-163.28,-177.75,1,100570_1_19990102,100570_1,1,63,Male,100570_1_19990102.nii.gz
1,106364,1.2.840.113654.2.55.16930993607533226779708566...,19990102,30.64,-95.96,-128.59,1,106364_1_19990102,106364_1,1,67,Female,106364_1_19990102.nii.gz
2,127302,1.2.840.113654.2.55.13728437630416839550344528...,20000102,-82.53,-27.85,-209.46,1,127302_1_20000102,127302_1,1,61,Male,127302_1_20000102.nii.gz
3,128899,1.2.840.113654.2.55.40415300708683206534849072...,20010102,-38.69,-8.78,-39.43,1,128899_1_20010102,128899_1,1,64,Male,128899_1_20010102.nii.gz
4,215816,1.3.6.1.4.1.14519.5.2.1.7009.9004.241362276021...,19990102,32.89,-37.98,-100.04,1,215816_1_19990102,215816_1,1,69,Female,215816_1_19990102.nii.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6158,123244,1.2.840.113654.2.55.96785330641884045479008848...,20010102,50.88,55.09,1816.38,1,123244_1_20010102,123244_1,0,73,Female,123244_1_20010102.nii.gz
6159,104196,1.2.840.113654.2.55.67476228289351741200494264...,20000102,84.01,-94.42,1106.82,1,104196_1_20000102,104196_1,0,58,Female,104196_1_20000102.nii.gz
6160,104196,1.2.840.113654.2.55.24806580836056963319112187...,20010102,70.29,-85.76,1904.06,1,104196_1_20010102,104196_1,0,59,Female,104196_1_20010102.nii.gz
6161,204005,1.3.6.1.4.1.14519.5.2.1.7009.9004.291214500058...,19990102,128.81,11.64,-256.59,1,204005_1_19990102,204005_1,0,59,Male,204005_1_19990102.nii.gz


In [6]:
all_participants_ct

,cen,dataset_version,elig,ineligible,pid,rndgroup,study,age,educat,ethnic,...,finaldeathlc,hasdcf,ndicd,evp_revr,evpcert,evpdeath,evpdirect,evpincomplete,evpsel,evpsent
0,AF,2011.02.03/04.07.22,2,NaN,100002,1,1,66,3,2,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AF,2011.02.03/04.07.22,2,NaN,100004,1,1,60,5,2,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AA,2011.02.03/04.07.22,2,NaN,100005,1,1,64,2,2,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AO,2011.02.03/04.07.22,2,NaN,100009,1,1,55,7,2,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AO,2011.02.03/04.07.22,2,NaN,100010,1,1,68,5,2,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26717,AB,2011.02.03/04.07.22,2,NaN,218888,1,3,58,5,2,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26718,AE,2011.02.03/04.07.22,2,NaN,218890,1,2,73,3,2,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26719,BF,2011.02.03/04.07.22,2,NaN,218892,1,2,56,5,2,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26720,AJ,2011.02.03/04.07.22,2,NaN,218893,1,3,69,4,2,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
all_participants[[PATIENT_RACE_COLMN_NAME,

race
White                                                         48549
Black or African-American                                      2376
Asian                                                          1095
More than one race                                              679
Native Hawaiian or Other Pacific Islander                       193
American Indian or Alaskan Native                               190
Participant refused to answer                                   109
Unknown/ decline to answer                                       81
Missing - no response                                             3
Missing - form was submitted and the answer was left blank        2
Name: count, dtype: int64

In [28]:
###################################################--Libaries---#######################################################

import os
import glob
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from tabulate import tabulate
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 8})



## Functions ###

def percentage_and_count(df,colmn):
        ###############--Race-#######
    # Perform value counts on the 'race_decoce' column and normalize the counts
    total_counts            = df[colmn].value_counts(normalize=False)
    total_percentage        = df[colmn].value_counts(normalize=True)

    
    count_number         = total_counts.values
    count_percentage     = total_percentage.values
    count_keys           = total_counts.keys()

    # Extract the keys (unique values) from the value counts result
    # Create a list of lists with keys and counts
    data = [[key, count,count2] for key, count,count2 in zip(count_keys,count_number,count_percentage)]
    headers = [colmn,"#", "(%)"]
    print(tabulate(data,headers, tablefmt="github"))     






###################################################--Libaries---#######################################################

import os
import glob
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from tabulate import tabulate
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 8})

def percentage_and_count(df,colmn):
        ###############--Race-#######
    # Perform value counts on the 'race_decoce' column and normalize the counts
    total_counts            = df[colmn].value_counts(normalize=False)
    total_percentage        = df[colmn].value_counts(normalize=True)

    
    count_number         = total_counts.values
    count_percentage     = total_percentage.values
    count_keys           = total_counts.keys()

    # Extract the keys (unique values) from the value counts result
    # Create a list of lists with keys and counts
    data = [[key, count,count2] for key, count,count2 in zip(count_keys,count_number,count_percentage)]
    headers = [colmn,"#", "(%)"]
    print(tabulate(data,headers, tablefmt="github"))     


################################################## --Directories---#######################################################
DF_SCR_PATH       ='/NAS/shared_data/VNLST/NLST_RAW_MetaData/'




#####################################################--Filtering CONDITIONS---#######################################################

FILTER_BY_YEAR            = False # Filter by screening year
FILTER_BY_GENDER          = False  # Filter by screening year
FILTER_BY_ABNORMALITIES   = False # Filter by screening year
PLOT                      = False


######################################################## COLMN NAMES ##################################################

PATIENT_ID_COLMN_NAME                      = 'pid'
PATIENT_GENDER_COLMN_NAME                  = 'gender'
PATIENT_AGE_COLMN_NAME                     = 'age'
PATIENT_RACE_COLMN_NAME                    = 'race'
PATIENT_STUDYYEAR_COLMN_NAME               ='study_yr'
PATIENT_ABNORNALITY_DESCRIPTION_COLMN_NAME = 'sct_ab_desc'



CANCER_GRP_COLMM_NAME                       = 'scr_group'
CANCER_SCR_COLMM_NAME                       = 'can_scr'
CANCER_RPT_LINK_COLMM_NAME                  = 'canc_rpt_link' # Is the diagnosis of lung cancer associated with a positive screen?
ANY_REPORTED_CANCER_COLMM_NAME              = 'anyscr_has_nodule'
LUNG_CANCER_COLMM_NAME                      = 'lung_cancer'                                                   
NODULE_LOC_COLMM_NAME                      = "sct_epi_loc"
NODULE_MARGIN_COLMM_NAME                   = "sct_margins"
NODULE_ATT_COLMM_NAME                      = "sct_pre_att"


ABNORNALITIES_TO_CONDIDER = [51]


#####################################################--Decoder---########################################################

GENDER_DECODER    = {1 :"male",2 :"female"}

ABNORNALITY_DESCRIPTION_DECODER = {51 : "Non-calcified nodule or mass (opacity >= 4 mm diameter)",
                      52 :"Non-calcified micronodule(s) (opacity < 4 mm diameter)",
                      53 :"Benign lung nodule(s) (benign calcification)",
                      54 :"Atelectasis, segmental or greater",
                      55 :"Pleural thickening or effusion",
                      56 :"Non-calcified hilar/mediastinal adenopathy or mass (>= 10 mm on short axis)",
                      57 :"Chest wall abnormality (bone destruction, metastasis, etc.)",
                      58 :"Consolidation",
                      59 :"Emphysema",
                      60 :"Significant cardiovascular abnormality",
                      61 :"Reticular/reticulonodular opacities, honeycombing, fibrosis, scar",
                      62 :"6 or more nodules, not suspicious for cancer (opacity >= 4 mm)",
                      63 :"Other potentially significant abnormality above the diaphragm",
                      64 :"Other potentially significant abnormality below the diaphragm",
                      65 :"Other minor abnormality noted"}


RACE_DECODER        = { 1:"White",
                        2:"Black or African-American",
                        3:"Asian",
                        4:"American Indian or Alaskan Native",
                        5:"Native Hawaiian or Other Pacific Islander",
                        6:"More than one race",
                        7:"Participant refused to answer",
                        9:"Missing data form - form is not expected to ever be completed",
                        96:"Missing - no response",
                        98:"Missing - form was submitted and the answer was left blank",
                        99:"Unknown/ decline to answer"}

RACE_DECODER        = { 1:"White",
                        2:"Black or African-American",
                        3:"Others",
                        4:"Others",
                        5:"Others",
                        6:"Others",
                        7:"Others",
                        9:"Others",
                        96:"Others",
                        98:"Others",
                        99:"Others"}





MARGIN_DECODER       = {1 :"Spiculated (Stellate)",
                        2 :"Smooth",
                        3 :"Poorly defined",
                        9 :"Unable to determine"}


ATTN_DECODER        ={1:"Soft Tissue",
                     2:"Ground glass",
                     3:"Mixed",
                     4:"Fluid/water",
                     6:"Fat",
                     7:"Other",
                     9:"Unable to determine"}

LOC_DECODER        ={1:"Right Upper Lobe",
                 2:"Right Middle Lobe",
                 3:"Right Lower Lobe",
                 4:"Left Upper Lobe",
                 5:"Lingula",
                 6:"Left Lower Lobe",
                 8:"Other"}


screening_group_decoder = {
    1: "Screen-detected cancer",
    2: "Has nodule(s)",
    3: "Screened, but no nodules",
    4: "Never screened",
    5: "Other lung cancer"
}


##==== cancer screening 
can_scr_decoder = {
    0: "No Cancer",
    1: "Positive Screen",
    2: "Negative Screen",
    3: "Missed Screen",
    4: "Post Screening"
}


ethnic_decode = {1:"Hispanic",
                 2:"Not Hispanic",
                 7:"Unknown/ decline to answer",
                 95:"Unknown/ decline to answer",
                 98:"Unknown/ decline to answer",
                 99:"Unknown/ decline to answer"}

#--------------------------------##


##====== All participants data ========#
all_participants                             = pd.read_csv(DF_SCR_PATH+'participant_d040722.csv')
all_participants[PATIENT_GENDER_COLMN_NAME]  = all_participants[PATIENT_GENDER_COLMN_NAME].map(GENDER_DECODER)
all_participants[PATIENT_RACE_COLMN_NAME]    = all_participants[PATIENT_RACE_COLMN_NAME].map(RACE_DECODER)
#Study arm| CT=1
all_participants_ct              = all_participants[all_participants['rndgroup']==1].reset_index(drop=True)
all_participants_ct['scr_group'] = all_participants_ct['scr_group'].map(screening_group_decoder)
all_participants_ct['can_scr']   = all_participants_ct['can_scr'].map(can_scr_decoder)
all_participants_ct['ethnic']   = all_participants_ct['ethnic'].map(ethnic_decode)


print('# Screening Result Grou')
print(all_participants_ct['scr_group'].value_counts())

print('\n')
print('# Result of screen associated with the first confirmed lung cancer diagnosis')
print(all_participants_ct['can_scr'].value_counts())

print('\n')
print('# Is the diagnosis of lung cancer associated with a positive screen?')
print(all_participants_ct['canc_rpt_link'].value_counts())

print('\n')
print('# Cancer confirmed year')
print(all_participants_ct['cancyr'].value_counts())

print('\n')
print('# Was a non-calcified nodule/mass >=4mm found on any screen?')
print(all_participants_ct['anyscr_has_nodule'].value_counts())

print('\n')
print('# Does the participant have a confirmed lung cancer diagnosis?')
print(all_participants_ct['lung_cancer'].value_counts())


              


/tmp/ipykernel_5176/2166433195.py:209: DtypeWarning: Columns (239,240,348) have mixed types. Specify dtype option on import or set low_memory=False.
  all_participants                             = pd.read_csv(DF_SCR_PATH+'participant_d040722.csv')


# Screening Result Grou
scr_group
Screened, but no nodules    15971
Has nodule(s)                9408
Screen-detected cancer        649
Other lung cancer             440
Never screened                254
Name: count, dtype: int64


# Result of screen associated with the first confirmed lung cancer diagnosis
can_scr
No Cancer          25633
Positive Screen      649
Post Screening       369
Negative Screen       44
Missed Screen         27
Name: count, dtype: int64


# Is the diagnosis of lung cancer associated with a positive screen?
canc_rpt_link
0    26038
1      684
Name: count, dtype: int64


# Cancer confirmed year
cancyr
0.0    297
2.0    237
1.0    186
5.0    133
4.0    107
3.0     70
6.0     58
7.0      1
Name: count, dtype: int64


# Was a non-calcified nodule/mass >=4mm found on any screen?
anyscr_has_nodule
0.0    16200
1.0    10257
Name: count, dtype: int64


# Does the participant have a confirmed lung cancer diagnosis?
lung_cancer
0    25633
1     1089
Name: count, dtype: 

In [29]:
roi_pt = all_participants_ct[all_participants_ct['pid'].isin(df['PatientID'].tolist())]
roi_pt

,cen,dataset_version,elig,ineligible,pid,rndgroup,study,age,educat,ethnic,...,finaldeathlc,hasdcf,ndicd,evp_revr,evpcert,evpdeath,evpdirect,evpincomplete,evpsel,evpsent
6,AV,2011.02.03/04.07.22,2,NaN,100012,1,1,61,7,Not Hispanic,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,BD,2011.02.03/04.07.22,2,NaN,100035,1,1,55,4,Not Hispanic,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,AI,2011.02.03/04.07.22,2,NaN,100069,1,1,64,5,Not Hispanic,...,0.0,1,NaN,1.0,1.0,3.0,NaN,NaN,1.0,1.0
47,AF,2011.02.03/04.07.22,2,NaN,100095,1,1,64,5,Not Hispanic,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,BG,2011.02.03/04.07.22,2,NaN,100150,1,1,56,4,Not Hispanic,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26671,AS,2011.02.03/04.07.22,2,NaN,218781,1,2,61,7,Not Hispanic,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26690,AM,2011.02.03/04.07.22,2,NaN,218817,1,3,60,3,Not Hispanic,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26693,AZ,2011.02.03/04.07.22,2,NaN,218829,1,3,59,6,Not Hispanic,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26708,BF,2011.02.03/04.07.22,2,NaN,218863,1,2,62,3,Not Hispanic,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
PATIENT_GENDER_COLMN_NAME,PATIENT_RACE_COLMN_NAME

('gender', 'race')

In [25]:
roi_pt[PATIENT_RACE_COLMN_NAME].value_counts()

race
White                        1975
Others                         76
Black or African-American      69
Name: count, dtype: int64

In [27]:
roi_pt[PATIENT_RACE_COLMN_NAME].value_counts(True)

race
White                        0.931604
Others                       0.035849
Black or African-American    0.032547
Name: proportion, dtype: float64

In [30]:
roi_pt['ethnic'].value_counts(True)

ethnic
Not Hispanic                  0.977358
Hispanic                      0.016038
Unknown/ decline to answer    0.006604
Name: proportion, dtype: float64

In [31]:
roi_pt['ethnic'].value_counts()

ethnic
Not Hispanic                  2072
Hispanic                        34
Unknown/ decline to answer      14
Name: count, dtype: int64

In [34]:
roi_pt['lung_cancer'].value_counts(True)

lung_cancer
0    0.842925
1    0.157075
Name: proportion, dtype: float64

In [35]:
roi_pt['lung_cancer'].value_counts()

lung_cancer
0    1787
1     333
Name: count, dtype: int64

In [36]:
1787+333

2120